In [2]:
import urllib.request, json
import pandas as pd
import argparse
import numpy as np
from influxdb import InfluxDBClient

In [2]:
with urllib.request.urlopen("https://data.cdc.gov/api/views/bi63-dtpu/rows.json?accessType=DOWNLOAD") as url:data = json.loads(url.read().decode())

In [3]:
parsed= data['data']

In [4]:
fined=[]
for i in parsed:
    fined.append(i[8:])

In [6]:
from IPython.display import display, HTML
df=pd.DataFrame(fined, columns=["Year", "113_Cause_Name", "Cause_Name", "State", "Deaths", "Age-adjusted_Death_Rate"])
del df["113_Cause_Name"]
df.to_csv("deaths.csv",index=False)
display(df)

,Year,Cause_Name,State,Deaths,Age-adjusted_Death_Rate
0,1999,Unintentional Injuries,Alabama,2313,52.2
1,1999,Unintentional Injuries,Alaska,294,55.9
2,1999,Unintentional Injuries,Arizona,2214,44.8
3,1999,Unintentional Injuries,Arkansas,1287,47.6
4,1999,Unintentional Injuries,California,9198,28.7
5,1999,Unintentional Injuries,Colorado,1519,39
6,1999,Unintentional Injuries,Connecticut,1034,29.3
7,1999,Unintentional Injuries,Delaware,267,35.3
8,1999,Unintentional Injuries,District of Columbia,161,28.4
9,1999,Unintentional Injuries,Florida,5961,35.7


In [ ]:

def read_data():
    with open('C:/Users/Swarupa/Desktop/deaths.csv') as f: #Input the name of your csv file here
        
        return [x.split(',') for x in f.readlines()[1:]]
def read_header():
    with open('C:/Users/Swarupa/Desktop/deaths.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[:1]]

def main(host='localhost', port=8086):       #input your port number which has been set for influx
    """Instantiate the connection to the InfluxDB client."""
    user = 'Swarupa'
    password = ''
    dbname = 'SocialWeather'
    # Temporarily avoid line protocol time conversion issues #412, #426, #431.
    protocol = 'json'

    client = InfluxDBClient(host, port, user, password, dbname)

    print("Create DataFrame");
    b=read_header();
    length=len(b);
    narr=(b[-1])
    narr[-1]=narr[-1].split('\n',1)[0]
    a = read_data();
    print(a[0])
    for metric in a:
        influx_metric = [{
            'measurement': 'Leading_Causes_Of_Deaths',
            'tags': {
                'State': metric[2],
                'Cause_Name': metric[1],
                'Deaths': metric[3],
                'Age-adjusted_Death_Rate': metric[4].split('\n',1)[0]
            },
            'time':str(metric[0])+"-12-31"+"T00:00:00Z",
            'fields': {
                'State': metric[2],
                'Cause_Name': metric[1],
                'Deaths': metric[3],
                'Age-adjusted_Death_Rate': metric[4].split('\n',1)[0]
                
            }
        }]
        client.write_points(influx_metric)

    print("Read DataFrame")
    print(client.query("select * from Leading_Causes_Of_Deaths")) # all values are now without /n
    
if __name__ == '__main__':
    main()

Create DataFrame
['1999', 'Unintentional Injuries', 'Alabama', '2313', '52.2\n']


In [ ]:
#Done